In [23]:
import pandas as pd
import math
import numpy as np
import jieba
import re

## non-cs (monolingual) data

In [11]:
file = open('non_cs_transcripts_output.txt', 'r')
lines = []
for line in file.readlines(): 
    lines.append(line.split())

In [12]:
lines[-5]

['学校=1867360',
 '2',
 '-6.3702464',
 '后面=0',
 '1',
 '-7.709398',
 '还=0',
 '1',
 '-7.1670523',
 '一个=616846',
 '1',
 '-6.1604376',
 '</s>=2',
 '1',
 '-0.42033282',
 'Total:',
 '-27.827467',
 'OOV:',
 '2']

In [13]:
len(lines)

22777

In [14]:
lines = lines[:-4]

In [15]:
lines[-1]

['学校=1867360',
 '2',
 '-6.3702464',
 '后面=0',
 '1',
 '-7.709398',
 '还=0',
 '1',
 '-7.1670523',
 '一个=616846',
 '1',
 '-6.1604376',
 '</s>=2',
 '1',
 '-0.42033282',
 'Total:',
 '-27.827467',
 'OOV:',
 '2']

In [19]:
mean_probs = []

for line in lines:
    end_index = line.index('</s>=2')
    new_line = line[:end_index]
    vals = new_line[2::3]
    mean = 0
    for val in vals:
        mean += float(val)
    mean /= len(vals)
    mean_probs.append(mean)
        #surps.append(-float(val))

In [20]:
len(mean_probs) == len(lines)

True

In [43]:
base_10_mean_probs = mean_probs

In [22]:
df = pd.read_csv('seame_non_cs_data.csv')
df.head()

,Unnamed: 0,file,type,conversation,speaker,transcript_2015,filtered_transcript_2015,length_of_turn,language,mean word length,random word,random word length,random POS,mode POS,random word frequency
0,2,01NC02FBY_0101,Conversation,1,2,我们,['我们'],1,ZH,2.0,我们,2,2,2,8.519816876703524
1,3,01NC02FBY_0101,Conversation,1,2,进 [啊] 没有,"['进', '没有']",2,ZH,1.5,进,1,1,1,11.230889976662251
2,4,01NC02FBY_0101,Conversation,1,2,(ppl) 等一下 (ppb),"['等', '一下']",2,ZH,1.5,一下,2,2,1,13.208367276887378
3,9,01NC02FBY_0101,Conversation,1,2,我们 拿 多少 分 [啊] 到 最后,"['我们', '拿', '多少', '分', '到', '最后']",6,ZH,1.5,最后,2,2,1,11.972945663039544
4,10,01NC02FBY_0101,Conversation,1,2,算,['算'],1,ZH,1.0,算,1,1,1,13.708818971267016


In [23]:
random_word = list(df['random word'])
len(random_word) == len(lines)

True

In [39]:
random_probs = []
for i in range(len(lines)):
    line = lines[i]
    end_index = line.index('</s>=2')
    new_line = line[:end_index]
    words = new_line[0::3]
    vals = new_line[2::3]
    clean_words = []
    for word in words:
        end_index = word.index('=')
        word = word[:end_index]
        clean_words.append(word)
    for j in range(len(clean_words)):
        word = clean_words[j]
        if word == random_word[i]:
            random_probs.append(float(vals[j]))
            break
            
            

In [45]:
base_10_random_probs = random_probs

In [48]:
base_10_mean_probs[0:10]

[-7.3514414,
 -7.25074475,
 -6.98285485,
 -6.53805615,
 -7.3514414,
 -7.344206,
 -6.61994505,
 -4.34711565,
 -6.7163184000000005,
 -7.029250366666667]

In [47]:
base_10_random_probs[0:10]

[-7.3514414,
 -7.310274,
 -9.118763,
 -7.255239,
 -7.3514414,
 -7.344206,
 -6.0811954,
 -5.320925,
 -7.3514414,
 -6.953135]

In [49]:
raw_mean_probs = [10**val for val in base_10_mean_probs]
raw_random_probs = [10**val for val in base_10_random_probs]

In [51]:
base_2_mean_probs = [math.log2(val) for val in raw_mean_probs]
base_2_random_probs = [math.log2(val) for val in raw_random_probs]

In [52]:
base_2_mean_surps = [-val for val in base_2_mean_probs]
base_2_random_surps = [-val for val in base_2_random_probs]

In [53]:
df.head()

,Unnamed: 0,file,type,conversation,speaker,transcript_2015,filtered_transcript_2015,length_of_turn,language,mean word length,random word,random word length,random POS,mode POS,random word frequency
0,2,01NC02FBY_0101,Conversation,1,2,我们,['我们'],1,ZH,2.0,我们,2,2,2,8.519816876703524
1,3,01NC02FBY_0101,Conversation,1,2,进 [啊] 没有,"['进', '没有']",2,ZH,1.5,进,1,1,1,11.230889976662251
2,4,01NC02FBY_0101,Conversation,1,2,(ppl) 等一下 (ppb),"['等', '一下']",2,ZH,1.5,一下,2,2,1,13.208367276887378
3,9,01NC02FBY_0101,Conversation,1,2,我们 拿 多少 分 [啊] 到 最后,"['我们', '拿', '多少', '分', '到', '最后']",6,ZH,1.5,最后,2,2,1,11.972945663039544
4,10,01NC02FBY_0101,Conversation,1,2,算,['算'],1,ZH,1.0,算,1,1,1,13.708818971267016


In [54]:
df['mean surprisal'] = base_2_mean_surps
df['random word surprisal'] = base_2_random_surps
df.head()

,Unnamed: 0,file,type,conversation,speaker,transcript_2015,filtered_transcript_2015,length_of_turn,language,mean word length,random word,random word length,random POS,mode POS,random word frequency,mean surprisal,random word surprisal
0,2,01NC02FBY_0101,Conversation,1,2,我们,['我们'],1,ZH,2.0,我们,2,2,2,8.519816876703524,24.420960,24.420960
1,3,01NC02FBY_0101,Conversation,1,2,进 [啊] 没有,"['进', '没有']",2,ZH,1.5,进,1,1,1,11.230889976662251,24.086453,24.284205
2,4,01NC02FBY_0101,Conversation,1,2,(ppl) 等一下 (ppb),"['等', '一下']",2,ZH,1.5,一下,2,2,1,13.208367276887378,23.196542,30.291875
3,9,01NC02FBY_0101,Conversation,1,2,我们 拿 多少 分 [啊] 到 最后,"['我们', '拿', '多少', '分', '到', '最后']",6,ZH,1.5,最后,2,2,1,11.972945663039544,21.718952,24.101382
4,10,01NC02FBY_0101,Conversation,1,2,算,['算'],1,ZH,1.0,算,1,1,1,13.708818971267016,24.420960,24.420960


In [55]:
df.to_csv('seame_non_cs_data.csv')

## redoing with models trained on common voice, not wiki

In [24]:
file = open('non_cs_transcripts_cv_output.txt', 'r')
lines = []
for line in file.readlines(): 
    lines.append(line.split())

In [27]:
lines[-5]

['学校=1274',
 '2',
 '-3.3787766',
 '后面=14971',
 '1',
 '-6.2503185',
 '还=1166',
 '1',
 '-3.228797',
 '一个=54',
 '1',
 '-3.2993882',
 '</s>=2',
 '2',
 '-2.3320713',
 'Total:',
 '-18.489351',
 'OOV:',
 '0']

In [28]:
len(lines)

22777

In [29]:
lines = lines[:-4]

In [30]:
lines[-1]

['学校=1274',
 '2',
 '-3.3787766',
 '后面=14971',
 '1',
 '-6.2503185',
 '还=1166',
 '1',
 '-3.228797',
 '一个=54',
 '1',
 '-3.2993882',
 '</s>=2',
 '2',
 '-2.3320713',
 'Total:',
 '-18.489351',
 'OOV:',
 '0']

In [31]:
mean_probs = []

for line in lines:
    end_index = line.index('</s>=2')
    new_line = line[:end_index]
    vals = new_line[2::3]
    mean = 0
    for val in vals:
        mean += float(val)
    mean /= len(vals)
    mean_probs.append(mean)
        #surps.append(-float(val))

In [32]:
len(mean_probs) == len(lines)

True

In [33]:
base_10_mean_probs = mean_probs

In [34]:
df = pd.read_csv('seame_non_cs_data.csv')
df.head()

,Unnamed: 0,file,type,conversation,speaker,transcript_2015,filtered_transcript_2015,length_of_turn,language,mean word length,random word,random word length,random POS,mode POS,random word frequency,mean surprisal,random word surprisal,mean word frequency
0,0,01NC02FBY_0101,Conversation,1,2,我们,['我们'],1,ZH,2.0,我们,2,2,2,8.519816876703520,24.420960,24.420960,7.519817
1,1,01NC02FBY_0101,Conversation,1,2,进 [啊] 没有,"['进', '没有']",2,ZH,1.5,进,1,1,1,11.230889976662300,24.086453,24.284205,8.926648
2,2,01NC02FBY_0101,Conversation,1,2,(ppl) 等一下 (ppb),"['等', '一下']",2,ZH,1.5,一下,2,2,1,13.208367276887400,23.196542,30.291875,9.308506
3,3,01NC02FBY_0101,Conversation,1,2,我们 拿 多少 分 [啊] 到 最后,"['我们', '拿', '多少', '分', '到', '最后']",6,ZH,1.5,最后,2,2,1,11.972945663039500,21.718952,24.101382,8.752136
4,4,01NC02FBY_0101,Conversation,1,2,算,['算'],1,ZH,1.0,算,1,1,1,13.708818971267000,24.420960,24.420960,12.708819


In [35]:
random_word = list(df['random word'])
len(random_word) == len(lines)

True

In [36]:
random_probs = []
for i in range(len(lines)):
    line = lines[i]
    end_index = line.index('</s>=2')
    new_line = line[:end_index]
    words = new_line[0::3]
    vals = new_line[2::3]
    clean_words = []
    for word in words:
        end_index = word.index('=')
        word = word[:end_index]
        clean_words.append(word)
    for j in range(len(clean_words)):
        word = clean_words[j]
        if word == random_word[i]:
            random_probs.append(float(vals[j]))
            break
            
            

In [37]:
base_10_random_probs = random_probs

In [38]:
base_10_mean_probs[0:10]

[-2.7065263,
 -4.48635065,
 -2.24537185,
 -3.789525116666667,
 -4.6094823,
 -6.242366,
 -5.15039055,
 -4.83476865,
 -5.4709627,
 -4.378342549999999]

In [39]:
base_10_random_probs[0:10]

[-2.7065263,
 -5.635716,
 -0.8893737,
 -2.4360368,
 -4.6094823,
 -6.242366,
 -3.9987624,
 -3.673311,
 -6.943163,
 -3.017186]

In [40]:
raw_mean_probs = [10**val for val in base_10_mean_probs]
raw_random_probs = [10**val for val in base_10_random_probs]

In [41]:
base_2_mean_probs = [math.log2(val) for val in raw_mean_probs]
base_2_random_probs = [math.log2(val) for val in raw_random_probs]

In [42]:
base_2_mean_surps = [-val for val in base_2_mean_probs]
base_2_random_surps = [-val for val in base_2_random_probs]

In [43]:
df.head()

,Unnamed: 0,file,type,conversation,speaker,transcript_2015,filtered_transcript_2015,length_of_turn,language,mean word length,random word,random word length,random POS,mode POS,random word frequency,mean surprisal,random word surprisal,mean word frequency
0,0,01NC02FBY_0101,Conversation,1,2,我们,['我们'],1,ZH,2.0,我们,2,2,2,8.519816876703520,24.420960,24.420960,7.519817
1,1,01NC02FBY_0101,Conversation,1,2,进 [啊] 没有,"['进', '没有']",2,ZH,1.5,进,1,1,1,11.230889976662300,24.086453,24.284205,8.926648
2,2,01NC02FBY_0101,Conversation,1,2,(ppl) 等一下 (ppb),"['等', '一下']",2,ZH,1.5,一下,2,2,1,13.208367276887400,23.196542,30.291875,9.308506
3,3,01NC02FBY_0101,Conversation,1,2,我们 拿 多少 分 [啊] 到 最后,"['我们', '拿', '多少', '分', '到', '最后']",6,ZH,1.5,最后,2,2,1,11.972945663039500,21.718952,24.101382,8.752136
4,4,01NC02FBY_0101,Conversation,1,2,算,['算'],1,ZH,1.0,算,1,1,1,13.708818971267000,24.420960,24.420960,12.708819


In [44]:
df['mean cv surprisal'] = base_2_mean_surps
df['random word cv surprisal'] = base_2_random_surps
df.head()

,Unnamed: 0,file,type,conversation,speaker,transcript_2015,filtered_transcript_2015,length_of_turn,language,mean word length,random word,random word length,random POS,mode POS,random word frequency,mean surprisal,random word surprisal,mean word frequency,mean cv surprisal,random word cv surprisal
0,0,01NC02FBY_0101,Conversation,1,2,我们,['我们'],1,ZH,2.0,我们,2,2,2,8.519816876703520,24.420960,24.420960,7.519817,8.990886,8.990886
1,1,01NC02FBY_0101,Conversation,1,2,进 [啊] 没有,"['进', '没有']",2,ZH,1.5,进,1,1,1,11.230889976662300,24.086453,24.284205,8.926648,14.903334,18.721443
2,2,01NC02FBY_0101,Conversation,1,2,(ppl) 等一下 (ppb),"['等', '一下']",2,ZH,1.5,一下,2,2,1,13.208367276887400,23.196542,30.291875,9.308506,7.458964,2.954435
3,3,01NC02FBY_0101,Conversation,1,2,我们 拿 多少 分 [啊] 到 最后,"['我们', '拿', '多少', '分', '到', '最后']",6,ZH,1.5,最后,2,2,1,11.972945663039500,21.718952,24.101382,8.752136,12.588530,8.092339
4,4,01NC02FBY_0101,Conversation,1,2,算,['算'],1,ZH,1.0,算,1,1,1,13.708818971267000,24.420960,24.420960,12.708819,15.312369,15.312369


In [45]:
df.to_csv('seame_non_cs_data.csv')

## cs data (all translated to english)

In [62]:
csw_df = pd.read_csv('seame_with_eng2_zh2_translations.csv')

In [63]:
csw_df.columns

Index(['Unnamed: 0', 'Unnamed: 0.1', 'Unnamed: 0.1.1', 'Unnamed: 0.1.1.1',
       'Unnamed: 0.1.1.1.1', 'Unnamed: 0.1.1.1.1.1', 'Unnamed: 0.1.1.1.1.1.1',
       'file', 'type', 'transcript_2015', 'filtered_transcript_2015',
       'length_of_turn', 'start_of_code_switches',
       'length_of_code_switched_segments', 'direction_of_code_switches',
       'number_of_code_switches', 'intersentential_code_switch?',
       'type_of_code_switching', 'language', 'conversation', 'speaker',
       'marking', 'first_cs_word', 'first_cs_word_eng_pos', 'word_length_eng',
       'english_translation', 'first_cs_word_zh', 'word_length_zh',
       'first_cs_word_zh_pos', 'first_cs_word_eng_raw_freq',
       'first_cs_word_zh_raw_freq', 'first_cs_word_eng_freq',
       'first_cs_word_zh_freq', 'chinese_translation',
       'chinese_wbw_translation', 'english_wbw_translation'],
      dtype='object')

In [64]:
csw_df = csw_df[['file', 'type', 'conversation', 'speaker', 'filtered_transcript_2015', 'length_of_turn',
       'direction_of_code_switches', 'first_cs_word',
       'first_cs_word_eng_pos', 'word_length_eng', 'english_translation', 'english_wbw_translation', 'first_cs_word_zh', 'word_length_zh', 'first_cs_word_zh_pos',
       'first_cs_word_eng_raw_freq', 'first_cs_word_zh_raw_freq',
       'first_cs_word_eng_freq', 'first_cs_word_zh_freq',
       'chinese_translation', 'chinese_wbw_translation']]
csw_df.head()

,file,type,conversation,speaker,filtered_transcript_2015,length_of_turn,direction_of_code_switches,first_cs_word,first_cs_word_eng_pos,word_length_eng,...,english_wbw_translation,first_cs_word_zh,word_length_zh,first_cs_word_zh_pos,first_cs_word_eng_raw_freq,first_cs_word_zh_raw_freq,first_cs_word_eng_freq,first_cs_word_zh_freq,chinese_translation,chinese_wbw_translation
0,01NC01FBX_0101,Conversation,1,1,"['呃', '十一号', '十月', '哦', '我', '跟', '你', '讲', '一...",15,"['man > eng', 'eng > man']",but,2,3,...,"uh, on october 11th, i tell you a thing to tel...",但,1,2,791724336,48791767,8.81279875271622,9.104577,呃十一十月我跟讲讲一一一个一个个不出去,呃 十一号 十月 哦 我 跟 你 讲 一个 东西 但 不 可以 讲出去
1,01NC02FBY_0101,Conversation,1,2,"['十月', 'you', 'mean', 'finals']",4,['man > eng'],you,2,3,...,october you mean finals,你,1,2,950006304,25465651,8.549859863469642,10.042662,十月你是说决赛,十月 你 的 意思 是 决赛
2,01NC02FBY_0101,Conversation,1,2,"['不', '可以', '讲出去', '我们', '是不是', 'yah']",6,['man > eng'],yah,0,3,...,can't tell if we are yah,雅虎,2,0,131496,11734,21.368562320373794,21.126304,不可以讲我们不是,不 可以 讲出去 我们 是不是 雅虎
3,01NC01FBX_0101,Conversation,1,1,"['我', '跟', '你', '讲', '一个', '事', 'but', '不', '可...",30,"['man > eng', 'eng > man', 'man > eng', 'eng >...",but,2,3,...,"i tell you one thing but can't tell us, we hav...",但,1,2,791724336,48791767,8.81279875271622,9.104577,我我讲一一一个个一一一可以讲出去有有有有有进有有有有进我们我们我们我们第三第三拿拿拿拿拿拿拿...,我 跟 你 讲 一个 事 但 不 可以 讲出去 我们 有进 然后 我们 拿 第三 但是 先生...
4,01NC01FBX_0101,Conversation,1,1,"['因为', 'top', 'four']",3,['man > eng'],top,2,3,...,because top four,顶部,2,0,40791068,0,13.091471764630844,34.644711,因为前四名,因为 前 四名


In [67]:
without_todo = csw_df.replace('TODO', np.NaN)
without_todo = without_todo.dropna()
len(without_todo)

28333

In [68]:
without_todo.head()

,file,type,conversation,speaker,filtered_transcript_2015,length_of_turn,direction_of_code_switches,first_cs_word,first_cs_word_eng_pos,word_length_eng,...,english_wbw_translation,first_cs_word_zh,word_length_zh,first_cs_word_zh_pos,first_cs_word_eng_raw_freq,first_cs_word_zh_raw_freq,first_cs_word_eng_freq,first_cs_word_zh_freq,chinese_translation,chinese_wbw_translation
0,01NC01FBX_0101,Conversation,1,1,"['呃', '十一号', '十月', '哦', '我', '跟', '你', '讲', '一...",15,"['man > eng', 'eng > man']",but,2,3,...,"uh, on october 11th, i tell you a thing to tel...",但,1,2,791724336,48791767,8.81279875271622,9.104577,呃十一十月我跟讲讲一一一个一个个不出去,呃 十一号 十月 哦 我 跟 你 讲 一个 东西 但 不 可以 讲出去
1,01NC02FBY_0101,Conversation,1,2,"['十月', 'you', 'mean', 'finals']",4,['man > eng'],you,2,3,...,october you mean finals,你,1,2,950006304,25465651,8.549859863469642,10.042662,十月你是说决赛,十月 你 的 意思 是 决赛
2,01NC02FBY_0101,Conversation,1,2,"['不', '可以', '讲出去', '我们', '是不是', 'yah']",6,['man > eng'],yah,0,3,...,can't tell if we are yah,雅虎,2,0,131496,11734,21.368562320373794,21.126304,不可以讲我们不是,不 可以 讲出去 我们 是不是 雅虎
3,01NC01FBX_0101,Conversation,1,1,"['我', '跟', '你', '讲', '一个', '事', 'but', '不', '可...",30,"['man > eng', 'eng > man', 'man > eng', 'eng >...",but,2,3,...,"i tell you one thing but can't tell us, we hav...",但,1,2,791724336,48791767,8.81279875271622,9.104577,我我讲一一一个个一一一可以讲出去有有有有有进有有有有进我们我们我们我们第三第三拿拿拿拿拿拿拿...,我 跟 你 讲 一个 事 但 不 可以 讲出去 我们 有进 然后 我们 拿 第三 但是 先生...
4,01NC01FBX_0101,Conversation,1,1,"['因为', 'top', 'four']",3,['man > eng'],top,2,3,...,because top four,顶部,2,0,40791068,0,13.091471764630844,34.644711,因为前四名,因为 前 四名


In [74]:
en_trans = list(without_todo['english_translation'])
en_trans_wbw = list(without_todo['english_wbw_translation'])

In [75]:
len(en_trans) == len(en_trans_wbw)

True

In [76]:
file = open('en_translated_transcripts.txt','w')
for line in en_trans:
    file.write(line+"\n")
file.close()

In [77]:
file = open('en_translated_wbw_transcripts.txt','w')
for line in en_trans_wbw:
    file.write(line+"\n")
file.close()

In [172]:
file = open('en_translated_transcripts_output.txt', 'r')
lines = []
for line in file.readlines(): 
    lines.append(line.split())

In [173]:
lines = lines[:-4]

In [174]:
csw_word = list(without_todo['first_cs_word'])
len(csw_word) == len(lines)

True

In [175]:
base_10_probs = ['TODO'] * len(lines)
for i in range(len(lines)):
    line = lines[i]
    end_index = line.index('</s>=2')
    new_line = line[:end_index]
    words = new_line[0::3]
    vals = new_line[2::3]
    clean_words = []
    for word in words:
        end_index = word.index('=')
        word = word[:end_index]
        clean_words.append(word)
    for j in range(len(clean_words)):
        word = clean_words[j]
        if word == csw_word[i]:
            base_10_probs[i] = float(vals[j])
            break           

In [176]:
len(base_10_probs)

28333

In [177]:
base_10_probs.count('TODO')

7450

In [178]:
raw_probs = [10**val if type(val) is float else val for val in base_10_probs]
base_2_probs = [math.log2(val) if type(val) is float else val for val in raw_probs]
base_2_surps = [-val if type(val) is float else val for val in base_2_probs]

In [179]:
without_todo['cs word en surprisal'] = base_2_surps
without_todo.head()

,file,type,conversation,speaker,filtered_transcript_2015,length_of_turn,direction_of_code_switches,first_cs_word,first_cs_word_eng_pos,word_length_eng,...,first_cs_word_eng_raw_freq,first_cs_word_zh_raw_freq,first_cs_word_eng_freq,first_cs_word_zh_freq,chinese_translation,chinese_wbw_translation,cs word en surprisal,cs word en wbw surprisal,cs word zh surprisal,cs word zh wbw surprisal
0,01NC01FBX_0101,Conversation,1,1,"['呃', '十一号', '十月', '哦', '我', '跟', '你', '讲', '一...",15,"['man > eng', 'eng > man']",but,2,3,...,791724336,48791767,8.81279875271622,9.104577,呃十一十月我跟讲讲一一一个一个个不出去,呃 十一号 十月 哦 我 跟 你 讲 一个 东西 但 不 可以 讲出去,TODO,19.326366,TODO,19.326366
1,01NC02FBY_0101,Conversation,1,2,"['十月', 'you', 'mean', 'finals']",4,['man > eng'],you,2,3,...,950006304,25465651,8.549859863469642,10.042662,十月你是说决赛,十月 你 的 意思 是 决赛,12.242704,26.61868,26.61868,26.61868
2,01NC02FBY_0101,Conversation,1,2,"['不', '可以', '讲出去', '我们', '是不是', 'yah']",6,['man > eng'],yah,0,3,...,131496,11734,21.368562320373794,21.126304,不可以讲我们不是,不 可以 讲出去 我们 是不是 雅虎,26.242903,23.808432,TODO,23.808432
3,01NC01FBX_0101,Conversation,1,1,"['我', '跟', '你', '讲', '一个', '事', 'but', '不', '可...",30,"['man > eng', 'eng > man', 'man > eng', 'eng >...",but,2,3,...,791724336,48791767,8.81279875271622,9.104577,我我讲一一一个个一一一可以讲出去有有有有有进有有有有进我们我们我们我们第三第三拿拿拿拿拿拿拿...,我 跟 你 讲 一个 事 但 不 可以 讲出去 我们 有进 然后 我们 拿 第三 但是 先生...,9.770063,19.37198,TODO,19.37198
4,01NC01FBX_0101,Conversation,1,1,"['因为', 'top', 'four']",3,['man > eng'],top,2,3,...,40791068,0,13.091471764630844,34.644711,因为前四名,因为 前 四名,16.562366,TODO,TODO,TODO


In [180]:
file = open('en_translated_wbw_transcripts_output.txt', 'r')
lines = []
for line in file.readlines(): 
    lines.append(line.split())

In [181]:
lines = lines[:-4]

In [182]:
len(csw_word) == len(lines)

True

In [183]:
base_10_probs = ['TODO'] * len(lines)
for i in range(len(lines)):
    line = lines[i]
    end_index = line.index('</s>=2')
    new_line = line[:end_index]
    words = new_line[0::3]
    vals = new_line[2::3]
    clean_words = []
    for word in words:
        end_index = word.index('=')
        word = word[:end_index]
        clean_words.append(word)
    for j in range(len(clean_words)):
        word = clean_words[j]
        if word == csw_word[i]:
            base_10_probs[i] = float(vals[j])
            break           

In [184]:
len(base_10_probs)

28333

In [185]:
base_10_probs.count('TODO')

274

In [186]:
raw_probs = [10**val if type(val) is float else val for val in base_10_probs]
base_2_probs = [math.log2(val) if type(val) is float else val for val in raw_probs]
base_2_surps = [-val if type(val) is float else val for val in base_2_probs]

In [187]:
without_todo['cs word en wbw surprisal'] = base_2_surps
without_todo.head()

,file,type,conversation,speaker,filtered_transcript_2015,length_of_turn,direction_of_code_switches,first_cs_word,first_cs_word_eng_pos,word_length_eng,...,first_cs_word_eng_raw_freq,first_cs_word_zh_raw_freq,first_cs_word_eng_freq,first_cs_word_zh_freq,chinese_translation,chinese_wbw_translation,cs word en surprisal,cs word en wbw surprisal,cs word zh surprisal,cs word zh wbw surprisal
0,01NC01FBX_0101,Conversation,1,1,"['呃', '十一号', '十月', '哦', '我', '跟', '你', '讲', '一...",15,"['man > eng', 'eng > man']",but,2,3,...,791724336,48791767,8.81279875271622,9.104577,呃十一十月我跟讲讲一一一个一个个不出去,呃 十一号 十月 哦 我 跟 你 讲 一个 东西 但 不 可以 讲出去,TODO,12.609204,TODO,19.326366
1,01NC02FBY_0101,Conversation,1,2,"['十月', 'you', 'mean', 'finals']",4,['man > eng'],you,2,3,...,950006304,25465651,8.549859863469642,10.042662,十月你是说决赛,十月 你 的 意思 是 决赛,12.242704,12.242704,26.61868,26.61868
2,01NC02FBY_0101,Conversation,1,2,"['不', '可以', '讲出去', '我们', '是不是', 'yah']",6,['man > eng'],yah,0,3,...,131496,11734,21.368562320373794,21.126304,不可以讲我们不是,不 可以 讲出去 我们 是不是 雅虎,26.242903,26.887935,TODO,23.808432
3,01NC01FBX_0101,Conversation,1,1,"['我', '跟', '你', '讲', '一个', '事', 'but', '不', '可...",30,"['man > eng', 'eng > man', 'man > eng', 'eng >...",but,2,3,...,791724336,48791767,8.81279875271622,9.104577,我我讲一一一个个一一一可以讲出去有有有有有进有有有有进我们我们我们我们第三第三拿拿拿拿拿拿拿...,我 跟 你 讲 一个 事 但 不 可以 讲出去 我们 有进 然后 我们 拿 第三 但是 先生...,9.770063,6.443982,TODO,19.37198
4,01NC01FBX_0101,Conversation,1,1,"['因为', 'top', 'four']",3,['man > eng'],top,2,3,...,40791068,0,13.091471764630844,34.644711,因为前四名,因为 前 四名,16.562366,16.562366,TODO,TODO


## cs data (all translated to chinese)

In [78]:
zh_trans = list(without_todo['chinese_translation'])
zh_trans_wbw = list(without_todo['chinese_wbw_translation'])

In [79]:
len(zh_trans) == len(zh_trans_wbw)

True

In [80]:
file = open('zh_translated_transcripts.txt','w')
for line in zh_trans:
    file.write(line+"\n")
file.close()

In [81]:
file = open('zh_translated_wbw_transcripts.txt','w')
for line in zh_trans_wbw:
    file.write(line+"\n")
file.close()

In [131]:
tokenized_zh_trans = []
for sent in zh_trans:
    tokenized_zh_trans.append(' '.join((jieba.lcut(sent))))

In [142]:
file = open('zh_translated_transcripts.txt','w')
for line in tokenized_zh_trans:
    file.write(line+"\n")
file.close()

In [188]:
file = open('zh_translated_transcripts_output.txt', 'r')
lines = []
for line in file.readlines(): 
    lines.append(line.split())

In [189]:
lines = lines[:-4]

In [190]:
csw_word = list(without_todo['first_cs_word_zh'])
len(csw_word) == len(lines)

True

In [191]:
base_10_probs = ['TODO'] * len(lines)
for i in range(len(lines)):
    line = lines[i]
    end_index = line.index('</s>=2')
    new_line = line[:end_index]
    words = new_line[0::3]
    vals = new_line[2::3]
    clean_words = []
    for word in words:
        end_index = word.index('=')
        word = word[:end_index]
        clean_words.append(word)
    for j in range(len(clean_words)):
        word = clean_words[j]
        if word == csw_word[i]:
            base_10_probs[i] = float(vals[j])
            break           

In [192]:
len(base_10_probs)

28333

In [193]:
base_10_probs.count('TODO')

19390

In [194]:
raw_probs = [10**val if type(val) is float else val for val in base_10_probs]
base_2_probs = [math.log2(val) if type(val) is float else val for val in raw_probs]
base_2_surps = [-val if type(val) is float else val for val in base_2_probs]

In [195]:
without_todo['cs word zh surprisal'] = base_2_surps
without_todo.head()

,file,type,conversation,speaker,filtered_transcript_2015,length_of_turn,direction_of_code_switches,first_cs_word,first_cs_word_eng_pos,word_length_eng,...,first_cs_word_eng_raw_freq,first_cs_word_zh_raw_freq,first_cs_word_eng_freq,first_cs_word_zh_freq,chinese_translation,chinese_wbw_translation,cs word en surprisal,cs word en wbw surprisal,cs word zh surprisal,cs word zh wbw surprisal
0,01NC01FBX_0101,Conversation,1,1,"['呃', '十一号', '十月', '哦', '我', '跟', '你', '讲', '一...",15,"['man > eng', 'eng > man']",but,2,3,...,791724336,48791767,8.81279875271622,9.104577,呃十一十月我跟讲讲一一一个一个个不出去,呃 十一号 十月 哦 我 跟 你 讲 一个 东西 但 不 可以 讲出去,TODO,12.609204,TODO,19.326366
1,01NC02FBY_0101,Conversation,1,2,"['十月', 'you', 'mean', 'finals']",4,['man > eng'],you,2,3,...,950006304,25465651,8.549859863469642,10.042662,十月你是说决赛,十月 你 的 意思 是 决赛,12.242704,12.242704,26.61868,26.61868
2,01NC02FBY_0101,Conversation,1,2,"['不', '可以', '讲出去', '我们', '是不是', 'yah']",6,['man > eng'],yah,0,3,...,131496,11734,21.368562320373794,21.126304,不可以讲我们不是,不 可以 讲出去 我们 是不是 雅虎,26.242903,26.887935,TODO,23.808432
3,01NC01FBX_0101,Conversation,1,1,"['我', '跟', '你', '讲', '一个', '事', 'but', '不', '可...",30,"['man > eng', 'eng > man', 'man > eng', 'eng >...",but,2,3,...,791724336,48791767,8.81279875271622,9.104577,我我讲一一一个个一一一可以讲出去有有有有有进有有有有进我们我们我们我们第三第三拿拿拿拿拿拿拿...,我 跟 你 讲 一个 事 但 不 可以 讲出去 我们 有进 然后 我们 拿 第三 但是 先生...,9.770063,6.443982,TODO,19.37198
4,01NC01FBX_0101,Conversation,1,1,"['因为', 'top', 'four']",3,['man > eng'],top,2,3,...,40791068,0,13.091471764630844,34.644711,因为前四名,因为 前 四名,16.562366,16.562366,TODO,TODO


In [196]:
file = open('zh_translated_wbw_transcripts_output.txt', 'r')
lines = []
for line in file.readlines(): 
    lines.append(line.split())

In [197]:
lines = lines[:-4]

In [198]:
len(csw_word) == len(lines)

True

In [199]:
base_10_probs = ['TODO'] * len(lines)
for i in range(len(lines)):
    line = lines[i]
    end_index = line.index('</s>=2')
    new_line = line[:end_index]
    words = new_line[0::3]
    vals = new_line[2::3]
    clean_words = []
    for word in words:
        end_index = word.index('=')
        word = word[:end_index]
        clean_words.append(word)
    for j in range(len(clean_words)):
        word = clean_words[j]
        if word == csw_word[i]:
            base_10_probs[i] = float(vals[j])
            break           

In [200]:
len(base_10_probs)

28333

In [201]:
base_10_probs.count('TODO')

10662

In [202]:
raw_probs = [10**val if type(val) is float else val for val in base_10_probs]
base_2_probs = [math.log2(val) if type(val) is float else val for val in raw_probs]
base_2_surps = [-val if type(val) is float else val for val in base_2_probs]

In [203]:
without_todo['cs word zh wbw surprisal'] = base_2_surps
without_todo.head()

,file,type,conversation,speaker,filtered_transcript_2015,length_of_turn,direction_of_code_switches,first_cs_word,first_cs_word_eng_pos,word_length_eng,...,first_cs_word_eng_raw_freq,first_cs_word_zh_raw_freq,first_cs_word_eng_freq,first_cs_word_zh_freq,chinese_translation,chinese_wbw_translation,cs word en surprisal,cs word en wbw surprisal,cs word zh surprisal,cs word zh wbw surprisal
0,01NC01FBX_0101,Conversation,1,1,"['呃', '十一号', '十月', '哦', '我', '跟', '你', '讲', '一...",15,"['man > eng', 'eng > man']",but,2,3,...,791724336,48791767,8.81279875271622,9.104577,呃十一十月我跟讲讲一一一个一个个不出去,呃 十一号 十月 哦 我 跟 你 讲 一个 东西 但 不 可以 讲出去,TODO,12.609204,TODO,19.326366
1,01NC02FBY_0101,Conversation,1,2,"['十月', 'you', 'mean', 'finals']",4,['man > eng'],you,2,3,...,950006304,25465651,8.549859863469642,10.042662,十月你是说决赛,十月 你 的 意思 是 决赛,12.242704,12.242704,26.61868,26.61868
2,01NC02FBY_0101,Conversation,1,2,"['不', '可以', '讲出去', '我们', '是不是', 'yah']",6,['man > eng'],yah,0,3,...,131496,11734,21.368562320373794,21.126304,不可以讲我们不是,不 可以 讲出去 我们 是不是 雅虎,26.242903,26.887935,TODO,23.808432
3,01NC01FBX_0101,Conversation,1,1,"['我', '跟', '你', '讲', '一个', '事', 'but', '不', '可...",30,"['man > eng', 'eng > man', 'man > eng', 'eng >...",but,2,3,...,791724336,48791767,8.81279875271622,9.104577,我我讲一一一个个一一一可以讲出去有有有有有进有有有有进我们我们我们我们第三第三拿拿拿拿拿拿拿...,我 跟 你 讲 一个 事 但 不 可以 讲出去 我们 有进 然后 我们 拿 第三 但是 先生...,9.770063,6.443982,TODO,19.37198
4,01NC01FBX_0101,Conversation,1,1,"['因为', 'top', 'four']",3,['man > eng'],top,2,3,...,40791068,0,13.091471764630844,34.644711,因为前四名,因为 前 四名,16.562366,16.562366,TODO,TODO


In [205]:
without_todo_dropped = without_todo.replace('TODO', np.NaN)
without_todo_dropped = without_todo_dropped.dropna()
len(without_todo_dropped)

6171

In [207]:
without_todo.to_csv('seame_with_eng2_zh2_translations.csv')
without_todo_dropped.to_csv('seame_with_eng2_zh2_translations_no_todos.csv')

## redoing with models trained on common voice, not wiki

In [47]:
csw_df = pd.read_csv('seame_with_eng2_zh2_translations.csv')

In [48]:
csw_df.columns

Index(['Unnamed: 0', 'file', 'type', 'conversation', 'speaker',
       'filtered_transcript_2015', 'length_of_turn',
       'direction_of_code_switches', 'first_cs_word', 'first_cs_word_eng_pos',
       'word_length_eng', 'english_translation', 'english_wbw_translation',
       'first_cs_word_zh', 'word_length_zh', 'first_cs_word_zh_pos',
       'first_cs_word_eng_raw_freq', 'first_cs_word_zh_raw_freq',
       'first_cs_word_eng_freq', 'first_cs_word_zh_freq',
       'chinese_translation', 'chinese_wbw_translation',
       'cs word en surprisal', 'cs word en wbw surprisal',
       'cs word zh surprisal', 'cs word zh wbw surprisal'],
      dtype='object')

In [49]:
csw_df = csw_df[['file', 'type', 'conversation', 'speaker', 'filtered_transcript_2015', 'length_of_turn',
       'direction_of_code_switches', 'first_cs_word',
       'first_cs_word_eng_pos', 'word_length_eng', 'english_translation', 'english_wbw_translation', 'first_cs_word_zh', 'word_length_zh', 'first_cs_word_zh_pos',
       'first_cs_word_eng_raw_freq', 'first_cs_word_zh_raw_freq',
       'first_cs_word_eng_freq', 'first_cs_word_zh_freq',
       'chinese_translation', 'chinese_wbw_translation']]
csw_df.head()

,file,type,conversation,speaker,filtered_transcript_2015,length_of_turn,direction_of_code_switches,first_cs_word,first_cs_word_eng_pos,word_length_eng,...,english_wbw_translation,first_cs_word_zh,word_length_zh,first_cs_word_zh_pos,first_cs_word_eng_raw_freq,first_cs_word_zh_raw_freq,first_cs_word_eng_freq,first_cs_word_zh_freq,chinese_translation,chinese_wbw_translation
0,01NC01FBX_0101,Conversation,1,1,"['呃', '十一号', '十月', '哦', '我', '跟', '你', '讲', '一...",15,"['man > eng', 'eng > man']",but,2,3,...,"uh, on october 11th, i tell you a thing to tel...",但,1,2,791724336,48791767,8.812799,9.104577,呃十一十月我跟讲讲一一一个一个个不出去,呃 十一号 十月 哦 我 跟 你 讲 一个 东西 但 不 可以 讲出去
1,01NC02FBY_0101,Conversation,1,2,"['十月', 'you', 'mean', 'finals']",4,['man > eng'],you,2,3,...,october you mean finals,你,1,2,950006304,25465651,8.549860,10.042662,十月你是说决赛,十月 你 的 意思 是 决赛
2,01NC02FBY_0101,Conversation,1,2,"['不', '可以', '讲出去', '我们', '是不是', 'yah']",6,['man > eng'],yah,0,3,...,can't tell if we are yah,雅虎,2,0,131496,11734,21.368562,21.126304,不可以讲我们不是,不 可以 讲出去 我们 是不是 雅虎
3,01NC01FBX_0101,Conversation,1,1,"['我', '跟', '你', '讲', '一个', '事', 'but', '不', '可...",30,"['man > eng', 'eng > man', 'man > eng', 'eng >...",but,2,3,...,"i tell you one thing but can't tell us, we hav...",但,1,2,791724336,48791767,8.812799,9.104577,我我讲一一一个个一一一可以讲出去有有有有有进有有有有进我们我们我们我们第三第三拿拿拿拿拿拿拿...,我 跟 你 讲 一个 事 但 不 可以 讲出去 我们 有进 然后 我们 拿 第三 但是 先生...
4,01NC01FBX_0101,Conversation,1,1,"['因为', 'top', 'four']",3,['man > eng'],top,2,3,...,because top four,顶部,2,0,40791068,0,13.091472,34.644711,因为前四名,因为 前 四名


In [50]:
without_todo = csw_df.replace('TODO', np.NaN)
without_todo = without_todo.dropna()
len(without_todo)

28333

In [51]:
without_todo.head()

,file,type,conversation,speaker,filtered_transcript_2015,length_of_turn,direction_of_code_switches,first_cs_word,first_cs_word_eng_pos,word_length_eng,...,english_wbw_translation,first_cs_word_zh,word_length_zh,first_cs_word_zh_pos,first_cs_word_eng_raw_freq,first_cs_word_zh_raw_freq,first_cs_word_eng_freq,first_cs_word_zh_freq,chinese_translation,chinese_wbw_translation
0,01NC01FBX_0101,Conversation,1,1,"['呃', '十一号', '十月', '哦', '我', '跟', '你', '讲', '一...",15,"['man > eng', 'eng > man']",but,2,3,...,"uh, on october 11th, i tell you a thing to tel...",但,1,2,791724336,48791767,8.812799,9.104577,呃十一十月我跟讲讲一一一个一个个不出去,呃 十一号 十月 哦 我 跟 你 讲 一个 东西 但 不 可以 讲出去
1,01NC02FBY_0101,Conversation,1,2,"['十月', 'you', 'mean', 'finals']",4,['man > eng'],you,2,3,...,october you mean finals,你,1,2,950006304,25465651,8.549860,10.042662,十月你是说决赛,十月 你 的 意思 是 决赛
2,01NC02FBY_0101,Conversation,1,2,"['不', '可以', '讲出去', '我们', '是不是', 'yah']",6,['man > eng'],yah,0,3,...,can't tell if we are yah,雅虎,2,0,131496,11734,21.368562,21.126304,不可以讲我们不是,不 可以 讲出去 我们 是不是 雅虎
3,01NC01FBX_0101,Conversation,1,1,"['我', '跟', '你', '讲', '一个', '事', 'but', '不', '可...",30,"['man > eng', 'eng > man', 'man > eng', 'eng >...",but,2,3,...,"i tell you one thing but can't tell us, we hav...",但,1,2,791724336,48791767,8.812799,9.104577,我我讲一一一个个一一一可以讲出去有有有有有进有有有有进我们我们我们我们第三第三拿拿拿拿拿拿拿...,我 跟 你 讲 一个 事 但 不 可以 讲出去 我们 有进 然后 我们 拿 第三 但是 先生...
4,01NC01FBX_0101,Conversation,1,1,"['因为', 'top', 'four']",3,['man > eng'],top,2,3,...,because top four,顶部,2,0,40791068,0,13.091472,34.644711,因为前四名,因为 前 四名


In [52]:
en_trans = list(without_todo['english_translation'])
en_trans_wbw = list(without_todo['english_wbw_translation'])

In [53]:
len(en_trans) == len(en_trans_wbw)

True

In [54]:
file = open('en_translated_cv_transcripts.txt','w')
for line in en_trans:
    file.write(line+"\n")
file.close()

In [55]:
file = open('en_translated_wbw_cv_transcripts.txt','w')
for line in en_trans_wbw:
    file.write(line+"\n")
file.close()

In [56]:
file = open('en_translated_transcripts_cv_output.txt', 'r')
lines = []
for line in file.readlines(): 
    lines.append(line.split())

In [57]:
lines = lines[:-4]

In [58]:
csw_word = list(without_todo['first_cs_word'])
len(csw_word) == len(lines)

True

In [59]:
base_10_probs = ['TODO'] * len(lines)
for i in range(len(lines)):
    line = lines[i]
    end_index = line.index('</s>=2')
    new_line = line[:end_index]
    words = new_line[0::3]
    vals = new_line[2::3]
    clean_words = []
    for word in words:
        end_index = word.index('=')
        word = word[:end_index]
        clean_words.append(word)
    for j in range(len(clean_words)):
        word = clean_words[j]
        if word == csw_word[i]:
            base_10_probs[i] = float(vals[j])
            break           

In [60]:
len(base_10_probs)

28333

In [61]:
base_10_probs.count('TODO')

7450

In [62]:
raw_probs = [10**val if type(val) is float else val for val in base_10_probs]
base_2_probs = [math.log2(val) if type(val) is float else val for val in raw_probs]
base_2_surps = [-val if type(val) is float else val for val in base_2_probs]

In [63]:
without_todo['cs word en cv surprisal'] = base_2_surps
without_todo.head()

,file,type,conversation,speaker,filtered_transcript_2015,length_of_turn,direction_of_code_switches,first_cs_word,first_cs_word_eng_pos,word_length_eng,...,first_cs_word_zh,word_length_zh,first_cs_word_zh_pos,first_cs_word_eng_raw_freq,first_cs_word_zh_raw_freq,first_cs_word_eng_freq,first_cs_word_zh_freq,chinese_translation,chinese_wbw_translation,cs word en cv surprisal
0,01NC01FBX_0101,Conversation,1,1,"['呃', '十一号', '十月', '哦', '我', '跟', '你', '讲', '一...",15,"['man > eng', 'eng > man']",but,2,3,...,但,1,2,791724336,48791767,8.812799,9.104577,呃十一十月我跟讲讲一一一个一个个不出去,呃 十一号 十月 哦 我 跟 你 讲 一个 东西 但 不 可以 讲出去,TODO
1,01NC02FBY_0101,Conversation,1,2,"['十月', 'you', 'mean', 'finals']",4,['man > eng'],you,2,3,...,你,1,2,950006304,25465651,8.549860,10.042662,十月你是说决赛,十月 你 的 意思 是 决赛,8.606572
2,01NC02FBY_0101,Conversation,1,2,"['不', '可以', '讲出去', '我们', '是不是', 'yah']",6,['man > eng'],yah,0,3,...,雅虎,2,0,131496,11734,21.368562,21.126304,不可以讲我们不是,不 可以 讲出去 我们 是不是 雅虎,18.510569
3,01NC01FBX_0101,Conversation,1,1,"['我', '跟', '你', '讲', '一个', '事', 'but', '不', '可...",30,"['man > eng', 'eng > man', 'man > eng', 'eng >...",but,2,3,...,但,1,2,791724336,48791767,8.812799,9.104577,我我讲一一一个个一一一可以讲出去有有有有有进有有有有进我们我们我们我们第三第三拿拿拿拿拿拿拿...,我 跟 你 讲 一个 事 但 不 可以 讲出去 我们 有进 然后 我们 拿 第三 但是 先生...,3.995471
4,01NC01FBX_0101,Conversation,1,1,"['因为', 'top', 'four']",3,['man > eng'],top,2,3,...,顶部,2,0,40791068,0,13.091472,34.644711,因为前四名,因为 前 四名,14.305564


In [64]:
file = open('en_translated_wbw_transcripts_cv_output.txt', 'r')
lines = []
for line in file.readlines(): 
    lines.append(line.split())

In [65]:
lines = lines[:-4]

In [66]:
len(csw_word) == len(lines)

True

In [67]:
base_10_probs = ['TODO'] * len(lines)
for i in range(len(lines)):
    line = lines[i]
    end_index = line.index('</s>=2')
    new_line = line[:end_index]
    words = new_line[0::3]
    vals = new_line[2::3]
    clean_words = []
    for word in words:
        end_index = word.index('=')
        word = word[:end_index]
        clean_words.append(word)
    for j in range(len(clean_words)):
        word = clean_words[j]
        if word == csw_word[i]:
            base_10_probs[i] = float(vals[j])
            break           

In [68]:
len(base_10_probs)

28333

In [69]:
base_10_probs.count('TODO')

274

In [70]:
raw_probs = [10**val if type(val) is float else val for val in base_10_probs]
base_2_probs = [math.log2(val) if type(val) is float else val for val in raw_probs]
base_2_surps = [-val if type(val) is float else val for val in base_2_probs]

In [71]:
without_todo['cs word en wbw cv surprisal'] = base_2_surps
without_todo.head()

,file,type,conversation,speaker,filtered_transcript_2015,length_of_turn,direction_of_code_switches,first_cs_word,first_cs_word_eng_pos,word_length_eng,...,word_length_zh,first_cs_word_zh_pos,first_cs_word_eng_raw_freq,first_cs_word_zh_raw_freq,first_cs_word_eng_freq,first_cs_word_zh_freq,chinese_translation,chinese_wbw_translation,cs word en cv surprisal,cs word en wbw cv surprisal
0,01NC01FBX_0101,Conversation,1,1,"['呃', '十一号', '十月', '哦', '我', '跟', '你', '讲', '一...",15,"['man > eng', 'eng > man']",but,2,3,...,1,2,791724336,48791767,8.812799,9.104577,呃十一十月我跟讲讲一一一个一个个不出去,呃 十一号 十月 哦 我 跟 你 讲 一个 东西 但 不 可以 讲出去,TODO,10.353134
1,01NC02FBY_0101,Conversation,1,2,"['十月', 'you', 'mean', 'finals']",4,['man > eng'],you,2,3,...,1,2,950006304,25465651,8.549860,10.042662,十月你是说决赛,十月 你 的 意思 是 决赛,8.606572,8.606572
2,01NC02FBY_0101,Conversation,1,2,"['不', '可以', '讲出去', '我们', '是不是', 'yah']",6,['man > eng'],yah,0,3,...,2,0,131496,11734,21.368562,21.126304,不可以讲我们不是,不 可以 讲出去 我们 是不是 雅虎,18.510569,18.510569
3,01NC01FBX_0101,Conversation,1,1,"['我', '跟', '你', '讲', '一个', '事', 'but', '不', '可...",30,"['man > eng', 'eng > man', 'man > eng', 'eng >...",but,2,3,...,1,2,791724336,48791767,8.812799,9.104577,我我讲一一一个个一一一可以讲出去有有有有有进有有有有进我们我们我们我们第三第三拿拿拿拿拿拿拿...,我 跟 你 讲 一个 事 但 不 可以 讲出去 我们 有进 然后 我们 拿 第三 但是 先生...,3.995471,7.45809
4,01NC01FBX_0101,Conversation,1,1,"['因为', 'top', 'four']",3,['man > eng'],top,2,3,...,2,0,40791068,0,13.091472,34.644711,因为前四名,因为 前 四名,14.305564,14.305564


In [72]:
zh_trans = list(without_todo['chinese_translation'])
zh_trans_wbw = list(without_todo['chinese_wbw_translation'])

In [73]:
len(zh_trans) == len(zh_trans_wbw)

True

In [74]:
file = open('zh_translated_transcripts_cv_output.txt', 'r')
lines = []
for line in file.readlines(): 
    lines.append(line.split())

In [75]:
lines = lines[:-4]

In [76]:
csw_word = list(without_todo['first_cs_word_zh'])
len(csw_word) == len(lines)

True

In [77]:
base_10_probs = ['TODO'] * len(lines)
for i in range(len(lines)):
    line = lines[i]
    end_index = line.index('</s>=2')
    new_line = line[:end_index]
    words = new_line[0::3]
    vals = new_line[2::3]
    clean_words = []
    for word in words:
        end_index = word.index('=')
        word = word[:end_index]
        clean_words.append(word)
    for j in range(len(clean_words)):
        word = clean_words[j]
        if word == csw_word[i]:
            base_10_probs[i] = float(vals[j])
            break           

In [78]:
len(base_10_probs)

28333

In [79]:
base_10_probs.count('TODO')

19390

In [80]:
raw_probs = [10**val if type(val) is float else val for val in base_10_probs]
base_2_probs = [math.log2(val) if type(val) is float else val for val in raw_probs]
base_2_surps = [-val if type(val) is float else val for val in base_2_probs]

In [81]:
without_todo['cs word zh cv surprisal'] = base_2_surps
without_todo.head()

,file,type,conversation,speaker,filtered_transcript_2015,length_of_turn,direction_of_code_switches,first_cs_word,first_cs_word_eng_pos,word_length_eng,...,first_cs_word_zh_pos,first_cs_word_eng_raw_freq,first_cs_word_zh_raw_freq,first_cs_word_eng_freq,first_cs_word_zh_freq,chinese_translation,chinese_wbw_translation,cs word en cv surprisal,cs word en wbw cv surprisal,cs word zh cv surprisal
0,01NC01FBX_0101,Conversation,1,1,"['呃', '十一号', '十月', '哦', '我', '跟', '你', '讲', '一...",15,"['man > eng', 'eng > man']",but,2,3,...,2,791724336,48791767,8.812799,9.104577,呃十一十月我跟讲讲一一一个一个个不出去,呃 十一号 十月 哦 我 跟 你 讲 一个 东西 但 不 可以 讲出去,TODO,10.353134,TODO
1,01NC02FBY_0101,Conversation,1,2,"['十月', 'you', 'mean', 'finals']",4,['man > eng'],you,2,3,...,2,950006304,25465651,8.549860,10.042662,十月你是说决赛,十月 你 的 意思 是 决赛,8.606572,8.606572,15.416318
2,01NC02FBY_0101,Conversation,1,2,"['不', '可以', '讲出去', '我们', '是不是', 'yah']",6,['man > eng'],yah,0,3,...,0,131496,11734,21.368562,21.126304,不可以讲我们不是,不 可以 讲出去 我们 是不是 雅虎,18.510569,18.510569,TODO
3,01NC01FBX_0101,Conversation,1,1,"['我', '跟', '你', '讲', '一个', '事', 'but', '不', '可...",30,"['man > eng', 'eng > man', 'man > eng', 'eng >...",but,2,3,...,2,791724336,48791767,8.812799,9.104577,我我讲一一一个个一一一可以讲出去有有有有有进有有有有进我们我们我们我们第三第三拿拿拿拿拿拿拿...,我 跟 你 讲 一个 事 但 不 可以 讲出去 我们 有进 然后 我们 拿 第三 但是 先生...,3.995471,7.45809,TODO
4,01NC01FBX_0101,Conversation,1,1,"['因为', 'top', 'four']",3,['man > eng'],top,2,3,...,0,40791068,0,13.091472,34.644711,因为前四名,因为 前 四名,14.305564,14.305564,TODO


In [82]:
file = open('zh_translated_wbw_transcripts_cv_output.txt', 'r')
lines = []
for line in file.readlines(): 
    lines.append(line.split())

In [83]:
lines = lines[:-4]

In [84]:
len(csw_word) == len(lines)

True

In [85]:
base_10_probs = ['TODO'] * len(lines)
for i in range(len(lines)):
    line = lines[i]
    end_index = line.index('</s>=2')
    new_line = line[:end_index]
    words = new_line[0::3]
    vals = new_line[2::3]
    clean_words = []
    for word in words:
        end_index = word.index('=')
        word = word[:end_index]
        clean_words.append(word)
    for j in range(len(clean_words)):
        word = clean_words[j]
        if word == csw_word[i]:
            base_10_probs[i] = float(vals[j])
            break           

In [86]:
len(base_10_probs)

28333

In [87]:
base_10_probs.count('TODO')

10662

In [88]:
raw_probs = [10**val if type(val) is float else val for val in base_10_probs]
base_2_probs = [math.log2(val) if type(val) is float else val for val in raw_probs]
base_2_surps = [-val if type(val) is float else val for val in base_2_probs]

In [89]:
without_todo['cs word zh wbw cv surprisal'] = base_2_surps
without_todo.head()

,file,type,conversation,speaker,filtered_transcript_2015,length_of_turn,direction_of_code_switches,first_cs_word,first_cs_word_eng_pos,word_length_eng,...,first_cs_word_eng_raw_freq,first_cs_word_zh_raw_freq,first_cs_word_eng_freq,first_cs_word_zh_freq,chinese_translation,chinese_wbw_translation,cs word en cv surprisal,cs word en wbw cv surprisal,cs word zh cv surprisal,cs word zh wbw cv surprisal
0,01NC01FBX_0101,Conversation,1,1,"['呃', '十一号', '十月', '哦', '我', '跟', '你', '讲', '一...",15,"['man > eng', 'eng > man']",but,2,3,...,791724336,48791767,8.812799,9.104577,呃十一十月我跟讲讲一一一个一个个不出去,呃 十一号 十月 哦 我 跟 你 讲 一个 东西 但 不 可以 讲出去,TODO,10.353134,TODO,8.311926
1,01NC02FBY_0101,Conversation,1,2,"['十月', 'you', 'mean', 'finals']",4,['man > eng'],you,2,3,...,950006304,25465651,8.549860,10.042662,十月你是说决赛,十月 你 的 意思 是 决赛,8.606572,8.606572,15.416318,15.416318
2,01NC02FBY_0101,Conversation,1,2,"['不', '可以', '讲出去', '我们', '是不是', 'yah']",6,['man > eng'],yah,0,3,...,131496,11734,21.368562,21.126304,不可以讲我们不是,不 可以 讲出去 我们 是不是 雅虎,18.510569,18.510569,TODO,18.372867
3,01NC01FBX_0101,Conversation,1,1,"['我', '跟', '你', '讲', '一个', '事', 'but', '不', '可...",30,"['man > eng', 'eng > man', 'man > eng', 'eng >...",but,2,3,...,791724336,48791767,8.812799,9.104577,我我讲一一一个个一一一可以讲出去有有有有有进有有有有进我们我们我们我们第三第三拿拿拿拿拿拿拿...,我 跟 你 讲 一个 事 但 不 可以 讲出去 我们 有进 然后 我们 拿 第三 但是 先生...,3.995471,7.45809,TODO,14.203228
4,01NC01FBX_0101,Conversation,1,1,"['因为', 'top', 'four']",3,['man > eng'],top,2,3,...,40791068,0,13.091472,34.644711,因为前四名,因为 前 四名,14.305564,14.305564,TODO,TODO


In [90]:
without_todo_dropped = without_todo.replace('TODO', np.NaN)
without_todo_dropped = without_todo_dropped.dropna()
len(without_todo_dropped)

6171

In [93]:
without_todo.to_csv('seame_with_eng2_zh2_translations_cv_surps.csv')
without_todo_dropped.to_csv('seame_with_eng2_zh2_translations_no_todos_cv_surps.csv')

## common voice data 

In [10]:
common_voice_zh = []
with open("common_voice_zh_transcripts.txt", "r") as f:
    common_voice_zh = [(line[:-1]) for line in f.readlines()]

In [11]:
common_voice_zh

['参加了缅甸战役的诸场作战。',
 '国民政府统一中国，实施训政。',
 '圣艾智德堂与毗邻的前加尔默罗会修道院为圣艾智德团体的总部。',
 '寿春公主和傅忠生有一个独生子傅彦。',
 '外在动机可以分成下列四项',
 '今天超市有促销活动。',
 '他们的存在是为了减少反社会行为和防止犯罪活动。',
 '当塞人口变化图示',
 '其多生活于浅海滩涂以及河口咸淡水或淡水中。',
 '你可以把水杯拿来吗？',
 '朗代韦内克。',
 '隘寮溪被称为排湾文明的母亲河。',
 '《我的世界》是一款沙盒游戏。',
 '髀关穴是属于足阳明胃经的腧穴。',
 '他认为文学作品应该也被用来唤醒青年对社会的关心与敏感。',
 '房东和邻居的人品有问题',
 '明代的史册中已经有水族的名称了。',
 '邓愈晋升为江西行省右丞。',
 '乾隆十年三月卒。',
 '尖刺异寄居蟹为寄居蟹科异寄居蟹属下的一个种。',
 '现任中国文化部老干部书画学会常务副会长等职。',
 '莫戈永山脉的绝大部分坐落于中的。',
 '赌场讨债更加兇狠、更加恶劣',
 '高遵裕侄。',
 '林勇气是一名出生于日本兵库县川西市的女子射箭运动员。',
 '这个阶段恒星的生命期可能最多只有一千万年。',
 '开发东北航线的初期动机是经济性的。',
 '碧玉妆成一树高，万条垂下绿丝绦',
 '福隆省是越南曾经设立的一个省。',
 '苏联政府邀请国际组织建立整个鞑靼斯坦共和国的紧急供餐点。',
 '葎草蚜为常蚜科蚜属下的一个种。',
 '台湾是中国不可分割的一部分。',
 '日期可以帮助我们表示一个指定的日子。',
 '这在射电天文学诞生的初期严重阻碍了射电望远镜的发展。',
 '骨灰坛安置于美国纽约海德公园附近庄严寺内。',
 '我今天早上吃了一个鸡蛋。',
 '我忘了里面是怎么写的了。',
 '约瑟夫斯指出艾赛尼是当时犹太教的三大派系之一。',
 '是一款由科乐美和公司制作的动作类电子游戏。',
 '大部分国际学生均住在学校提供的寄住家庭。',
 '儿子李桃杖承袭了清渊县侯。',
 '绡蝶属是蛱蝶科斑蝶亚科绡蝶族中的一个属。',
 '故事发生在超高度文明迎向衰亡的近未来。',
 '你不能走回头路',
 '时任虢州刺史的满存率军奔赴行在。',
 '都準备好了吧',
 '中文维基百科的编辑者来自于世界各地',
 '这是

In [12]:
tokenized_common_voice = []
for sent in common_voice_zh:
    tokenized_common_voice.append(' '.join((jieba.lcut(sent))))

Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/1p/_f5vztgn53n2ctrgr8fsk3b80000gn/T/jieba.cache
Loading model cost 0.532 seconds.
Prefix dict has been built successfully.


In [13]:
file = open('common_voice_zh_tokenized_transcripts.txt','w')
for line in tokenized_common_voice:
    file.write(line+"\n")
file.close()

In [14]:
len(tokenized_common_voice)

3576

In [15]:
common_voice_en = []
with open("common_voice_en_transcripts.txt", "r") as f:
    common_voice_en = [(line[:-1]) for line in f.readlines()]

In [16]:
len(common_voice_en)

35733

In [94]:
file = open('common_voice_en_sample_output.txt', 'r')
lines = []
for line in file.readlines(): 
    lines.append(line.split())

In [95]:
lines = lines[:-4]

In [96]:
base_10_probs = []
for i in range(len(lines)):
    line = lines[i]
    try:
        end_index = line.index('</s>=2')
        new_line = line[:end_index]
        #words = new_line[0::3]
        vals = new_line[2::3]
        float_vals = [float(val) for val in vals]
        base_10_probs.extend(float_vals)
    except:
        new_line = line
        vals = new_line[2::3]
        float_vals = [float(val) for val in vals]
        base_10_probs.extend(float_vals)
            

In [97]:
len(base_10_probs)

388593

In [98]:
raw_probs = [10**val if type(val) is float else val for val in base_10_probs]
base_2_probs = [math.log2(val) if type(val) is float else val for val in raw_probs]
base_2_surps = [-val if type(val) is float else val for val in base_2_probs]

In [101]:
file = open('common_voice_en_sample_surprisals.txt','w')
for line in base_2_surps:
    file.write(str(line)+"\n")
file.close()

In [75]:
np.mean(base_2_surps) #mean surprisal of common voice english

12.28306594359866

In [3]:
common_voice_zh = []
with open("common_voice_zh_transcripts_corpus_15.0.txt", "r") as f:
    common_voice_zh = [(line[:-1]) for line in f.readlines()]

In [4]:
common_voice_zh

['目前正在爆发的兹卡病毒疫情引发运动员和游客的担忧。',
 '为松本市的。',
 '饭山站铁路车站。',
 '而日清贸易研究所是在他死后设立的东亚同文书院的前身。',
 '皮纹检测是近期时兴的一门学科。',
 '是桂林市重点中学之一。',
 '芝麻三壳灰介为半花介科三壳灰介属下的一个种。',
 '为晚会设计了新的装饰和舞台。',
 '瓦雷姆区为比利时列日省辖下的一个区。',
 '这和印度政府对缅甸的抗议和协助印度侨民转移资产的行动形成了鲜明对比。',
 '马扎尔人的到来结束了此一地区的纷争。',
 '城门校尉岑起举荐马融。',
 '大战乡是中国浙江省仙居县所辖的一个镇。',
 '媒介效果研究受众研究',
 '其中一名学生是徐悲鸿先生谁后来成了好朋友。',
 '结果被达克托阻止了。',
 '沙尔拉。',
 '然后他们把这个文本文件和校验和发给所有参与者。',
 '该公告所述和原先的赛程出现分歧。',
 '普世圣公宗主教长会议是普世圣公宗各个教省主教长参加的例行会议。',
 '坪石街是二战时期中山大学临时所在地。',
 '每个原始州选举的参议员数量必须同等。',
 '乾隆二十九年置。',
 '瀼河乡是中国河南省平顶山市鲁山县下辖的一个乡。',
 '这部作品中的爱情三角恋就发生在英格兰中部的一座只有一间房子的狭小公寓中。',
 '芒西尼亚克人口变化图示',
 '西南交通大学教授。',
 '如江西高腔的各种演出仍以演历史戏为主。',
 '努朗人口变化图示',
 '我们应该先告诉谁？',
 '苏格兰则以小组渗透的方式进攻。',
 '茶卡高原鳅为鳅科高原鳅属的鱼类。',
 '香荚兰的香味源自其种荚里名为香草精的化合物。',
 '他与祖父一起葬于圣若望及保禄堂中。',
 '香港太空馆天文公园全天候开放。',
 '游戏源于陈星汉在南加州大学时期的一篇研究论文。',
 '主线任务只会按游戏主故事线顺序出现一般可在任何时刻进行。',
 '唐慧娟，江苏海门人，汉族，中国共产党党员。',
 '出生于静冈。',
 '但苏联驻哈尔滨总领事馆仍照常工作。',
 '第二天，男孩们去了罗比的家。',
 '日本国外出身的武士列表列举出生于日本国以外，并成为日本武士的人物。',
 '这天新雨，路上有泥。',
 '文森特·路德维希·佩尔西凯蒂，美国作曲家、音乐教育家。',
 '波尔多斯，是大仲马小说系列

In [5]:
tokenized_common_voice = []
for sent in common_voice_zh:
    tokenized_common_voice.append(' '.join((jieba.lcut(sent))))

Building prefix dict from the default dictionary ...
Dumping model to file cache /var/folders/1p/_f5vztgn53n2ctrgr8fsk3b80000gn/T/jieba.cache
Loading model cost 0.660 seconds.
Prefix dict has been built successfully.


In [6]:
tokenized_common_voice

['目前 正在 爆发 的 兹 卡 病毒 疫情 引发 运动员 和 游客 的 担忧 。',
 '为松 本市 的 。',
 '饭山站 铁路 车站 。',
 '而 日 清 贸易 研究所 是 在 他 死 后 设立 的 东亚 同 文书 院 的 前身 。',
 '皮纹 检测 是 近期 时兴 的 一门 学科 。',
 '是 桂林市 重点中学 之一 。',
 '芝麻 三壳 灰介为 半花 介科三壳 灰介 属下 的 一个 种 。',
 '为 晚会 设计 了 新 的 装饰 和 舞台 。',
 '瓦雷姆区 为 比利时 列日 省 辖下 的 一个 区 。',
 '这 和 印度政府 对 缅甸 的 抗议 和 协助 印度 侨民 转移 资产 的 行动 形成 了 鲜明对比 。',
 '马扎 尔人 的 到来 结束 了 此 一 地区 的 纷争 。',
 '城门 校尉 岑起 举荐 马融 。',
 '大战 乡 是 中国 浙江省 仙居县 所辖 的 一个 镇 。',
 '媒介 效果 研究 受众 研究',
 '其中 一名 学生 是 徐悲鸿 先生 谁 后来 成 了 好 朋友 。',
 '结果 被 达克 托 阻止 了 。',
 '沙尔拉 。',
 '然后 他们 把 这个 文本文件 和校验 和 发给 所有 参与者 。',
 '该 公告 所述 和 原先 的 赛程 出现分歧 。',
 '普世圣公 宗主 教长 会议 是 普世圣 公宗 各个 教省 主教 长 参加 的 例行 会议 。',
 '坪石 街 是 二战时期 中山大学 临时 所在地 。',
 '每个 原始 州 选举 的 参议员 数量 必须 同等 。',
 '乾隆 二十九年 置 。',
 '瀼 河乡 是 中国 河南省 平顶山市 鲁山县 下辖 的 一个 乡 。',
 '这部 作品 中 的 爱情 三角恋 就 发生 在 英格兰 中部 的 一座 只有 一间 房子 的 狭小 公寓 中 。',
 '芒西 尼亚 克 人口 变化 图示',
 '西南 交通 大学教授 。',
 '如 江西 高腔 的 各种 演出 仍 以演 历史 戏 为主 。',
 '努朗 人口 变化 图示',
 '我们 应该 先 告诉 谁 ？',
 '苏格兰 则 以 小组 渗透 的 方式 进攻 。',
 '茶卡 高原 鳅 为 鳅 科 高原 鳅 属 的 鱼类 。',
 '香荚兰 的 香味 源自 其种 荚 里 名为 香草 精 的 化合物

In [8]:
file = open('common_voice_zh_tokenized_transcripts_corpus_15.0.txt','w')
for line in tokenized_common_voice:
    file.write(line+"\n")
file.close()

In [102]:
file = open('common_voice_zh_sample_output.txt', 'r')
lines = []
for line in file.readlines(): 
    lines.append(line.split())

In [103]:
lines[-5]

['《=327',
 '2',
 '-4.596014',
 '原神=0',
 '1',
 '-6.072101',
 '》=332',
 '1',
 '-3.3788784',
 '是=27',
 '2',
 '-1.9240944',
 '由=398',
 '2',
 '-2.2545805',
 '米哈=0',
 '1',
 '-5.7335644',
 '游=41466',
 '1',
 '-5.431228',
 '开发=1939',
 '1',
 '-3.727352',
 '的=6',
 '2',
 '-0.5802105',
 '动作=2408',
 '2',
 '-3.6615672',
 '角色扮演=16795',
 '2',
 '-2.3167338',
 '游=41466',
 '1',
 '-5.847697',
 '</s>=2',
 '1',
 '-2.2317412',
 'Total:',
 '-47.755768',
 'OOV:',
 '2']

In [104]:
lines = lines[:-4]

In [105]:
base_10_probs = []
for i in range(len(lines)):
    line = lines[i]
    try:
        end_index = line.index('</s>=2')
        new_line = line[:end_index]
        #words = new_line[0::3]
        vals = new_line[2::3]
        float_vals = [float(val) for val in vals]
        base_10_probs.extend(float_vals)
    except:
#         new_line = line
#         vals = new_line[2::3]
#         float_vals = [float(val) for val in vals]
#         base_10_probs.extend(float_vals)
        print(i)
        pass
            

In [106]:
len(base_10_probs)

31519

In [107]:
raw_probs = [10**val if type(val) is float else val for val in base_10_probs]
base_2_probs = [math.log2(val) if type(val) is float else val for val in raw_probs]
base_2_surps = [-val if type(val) is float else val for val in base_2_probs]

In [108]:
file = open('common_voice_zh_sample_surprisals.txt','w')
for line in base_2_surps:
    file.write(str(line)+"\n")
file.close()

In [22]:
np.mean(base_2_surps) #mean surprisal of common voice chinese

4.572252086427525